In [5]:
from pandas.io.json import json_normalize
from urllib.parse import quote
from urllib.request import Request, urlopen
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np

In [6]:
cfdata = pd.read_csv('./data/cfdata.csv')

In [7]:
#cfdata = cfdata.drop(['Unnamed: 0'],axis = 1)
cfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24541 entries, 0 to 24540
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   24541 non-null  int64  
 1   APVPERMYMD   24541 non-null  object 
 2   TRDSTATEGBN  24541 non-null  int64  
 3   TRDSTATENM   24541 non-null  object 
 4   DCBYMD       8624 non-null   object 
 5   SITEAREA     23486 non-null  object 
 6   BPLCNM       24541 non-null  object 
 7   X            24541 non-null  float64
 8   Y            24541 non-null  float64
 9   region       24541 non-null  object 
 10  ADDR         24541 non-null  object 
 11  JIGA         24541 non-null  int64  
 12  at4          24541 non-null  int64  
 13  po3          24541 non-null  int64  
 14  num_unv      24541 non-null  int64  
 15  unv_name     24541 non-null  object 
 16  unv_dist     24541 non-null  object 
dtypes: float64(2), int64(6), object(9)
memory usage: 3.2+ MB


In [8]:
cfdata.ADDR[:18]

0      서울특별시 강동구 양재대로87길 11 
1       서울특별시 강동구 진황도로27길 23
2        서울특별시 강동구 구천면로 428 
3         서울특별시 강동구 강동대로 205
4       서울특별시 강동구 풍성로63길 84 
5         서울특별시 강동구 올림픽로 660
6          서울특별시 강동구 진황도로 74
7       서울특별시 강동구 풍성로63길 79 
8        서울특별시 강동구 고덕로83길 28
9        서울특별시 강동구 천호대로 1016
10       서울특별시 강동구 양재대로 1355
11       서울특별시 강동구 동남로81길 76
12        서울특별시 강동구 천호대로 991
13       서울특별시 강동구 천호대로 1092
14    서울특별시 강동구 양재대로128길 50 
15       서울특별시 강동구 천호대로 1005
16      서울특별시 강동구 상암로51길 46 
17    서울특별시 강동구 천호대로 1138-18
Name: ADDR, dtype: object

In [9]:
url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[24]
headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
result1 = json.loads(str(requests.get(url1, headers=headers).text))
result1

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [10]:
gu_list=[]
dong_list=[]
madd_list =[]
sadd_list = []

for i in range(len(cfdata.ADDR)):
    
    #카카오 API 주소검색
    
    url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[i]
    headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result1 = json.loads(str(requests.get(url1, headers=headers).text))
    
    if len(result1['documents']) == 0:    #주소검색이 되지 않는 주소일 시 건너뜀
        i += 1
        gu_list.append('0')
        dong_list.append('0')
        madd_list.append('0')
        sadd_list.append('0')
    else:
        addinfo = result1['documents'][0]['address']
        
        gu = addinfo['region_2depth_name']
        dong = addinfo['region_3depth_name']
        main_add = (4-len(addinfo['main_address_no'])) * '0' + addinfo['main_address_no']
        sub_add = (4-len(addinfo['sub_address_no'])) * '0' + addinfo['sub_address_no']
        
        gu_list.append(gu)
        dong_list.append(dong)
        madd_list.append(main_add)
        sadd_list.append(sub_add)
        

print(len(gu_list))

24541


In [11]:
madd_list.count('0')

284

In [14]:
import urllib

In [16]:
gu = quote('성동구')
dong = quote('성수동1가')

url='http://openapi.seoul.go.kr:8088/4d53677a5a726e6439346c526d4f50/json/IndividuallyPostedLandPriceService/1/5/'+gu+'/%20/%20/%20/%20/2020'
url1='http://openapi.seoul.go.kr:8088/4d53677a5a726e6439346c526d4f50/json/IndividuallyPostedLandPriceService/1/5/'+gu+'/'+dong+'/0656/0745/%20/2020'
# %20 : 공백
#구/동/본번/부번/공백/2020
response = urllib.request.urlopen(url1)
json_str = response.read().decode("utf-8")
json_object = json.loads(json_str)
json_str

'{"RESULT":{"CODE":"INFO-200","MESSAGE":"해당하는 데이터가 없습니다."}}'

In [23]:
date = pd.to_datetime(cfdata['DCBYMD'],format='%Y-%m-%d')
opdate = pd.to_datetime(cfdata['APVPERMYMD'],format='%Y-%m-%d')
cfdata['DCBYMD'] = date
cfdata['APVPERMYMD'] = opdate

cfdata.TRDSTATEGBN

0        3
1        3
2        3
3        3
4        3
        ..
24536    1
24537    1
24538    1
24539    1
24540    1
Name: TRDSTATEGBN, Length: 24541, dtype: int64

In [33]:
st_year = []

for i in range(len(cfdata)):
    if cfdata['TRDSTATEGBN'][i] == 3 :
        cl_year = cfdata['DCBYMD'][i].year
        st_year.append(str(cl_year))
    else:
        st_year.append('2020')

len(st_year)

24541

In [34]:
st_year[24531:24541]

['2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020']

In [35]:
all_jiga=[]


for i in range(len(gu_list)):

    gu = quote(gu_list[i])
    dong = quote(dong_list[i])
    url = 'http://openapi.seoul.go.kr:8088/4d53677a5a726e6439346c526d4f50/json/IndividuallyPostedLandPriceService/1/5/'+gu+'/'+dong+'/'+madd_list[i]+'/'+sadd_list[i]+'/%20/'+st_year[i]

    response = urllib.request.urlopen(url)
    json_str = response.read().decode("utf-8")

    if len(json_str) > 58 :
        json_object = json.loads(json_str)
        jiga = json_object['IndividuallyPostedLandPriceService']['row'][0]['JIGA']

        all_jiga.append(jiga)
        
    else:
        i += 1
        all_jiga.append('0')
        

print(len(all_jiga))

24541


In [49]:
a='%20'
urllib.parse.unquote(a)

' '

In [36]:
all_jiga.count('0')
#len(all_jiga)

459

In [37]:
cfdata['JIGA'] = all_jiga

In [38]:
cfdata = cfdata.drop(['Unnamed: 0'],axis = 1)
cfdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,BPLCNM,X,Y,region,ADDR,JIGA,at4,po3,num_unv,unv_name,unv_dist
0,2013-11-13,3,폐업,2017-10-11,22.53,탭플레이 커피,127.134306,37.525438,강동구,서울특별시 강동구 양재대로87길 11,6350000,6,8,1,한국체육대학교,662
1,2015-07-08,3,폐업,2017-07-10,22.00,아카페,127.131877,37.539416,강동구,서울특별시 강동구 진황도로27길 23,4831000,10,10,0,없음,x
2,2014-03-20,3,폐업,2017-07-17,74.21,CAFE TENUTO 테누토,127.146158,37.550130,강동구,서울특별시 강동구 구천면로 428,6596000,3,11,0,없음,x
3,2014-12-05,3,폐업,2017-06-02,337.19,할리스커피 올림픽공원북문점,127.129210,37.524394,강동구,서울특별시 강동구 강동대로 205,8360000,9,9,1,한국체육대학교,490
4,2013-01-03,3,폐업,2017-05-18,30.00,도우드림,127.140339,37.531725,강동구,서울특별시 강동구 풍성로63길 84,4420000,3,10,0,없음,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,카페 엘리,127.032416,37.485785,서초구,서울특별시 서초구 강남대로39길 15-7,0,1,12,2,횃불트리니티 신학대학원대학교,746
24537,2010-09-16,1,영업/정상,NaT,50.00,카페레즈,127.041496,37.481075,서초구,서울특별시 서초구 바우뫼로41길 8,8030000,6,9,3,서울외국어대학원대학교,235
24538,2010-05-27,1,영업/정상,NaT,142.50,마켓 비노플라워,126.996156,37.498874,서초구,서울특별시 서초구 사평대로 106,12190000,5,11,1,가톨릭대학교 성의교정,839
24539,2009-10-29,1,영업/정상,NaT,62.00,까페원더,127.039916,37.484342,서초구,서울특별시 서초구 바우뫼로41길 88,7904000,4,8,3,서울외국어대학원대학교,341


In [39]:
cfdata.to_csv('./data/cfdata.csv',encoding='utf-8')

In [40]:
cfdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,BPLCNM,X,Y,region,ADDR,JIGA,at4,po3,num_unv,unv_name,unv_dist
0,2013-11-13,3,폐업,2017-10-11,22.53,탭플레이 커피,127.134306,37.525438,강동구,서울특별시 강동구 양재대로87길 11,6350000,6,8,1,한국체육대학교,662
1,2015-07-08,3,폐업,2017-07-10,22.00,아카페,127.131877,37.539416,강동구,서울특별시 강동구 진황도로27길 23,4831000,10,10,0,없음,x
2,2014-03-20,3,폐업,2017-07-17,74.21,CAFE TENUTO 테누토,127.146158,37.550130,강동구,서울특별시 강동구 구천면로 428,6596000,3,11,0,없음,x
3,2014-12-05,3,폐업,2017-06-02,337.19,할리스커피 올림픽공원북문점,127.129210,37.524394,강동구,서울특별시 강동구 강동대로 205,8360000,9,9,1,한국체육대학교,490
4,2013-01-03,3,폐업,2017-05-18,30.00,도우드림,127.140339,37.531725,강동구,서울특별시 강동구 풍성로63길 84,4420000,3,10,0,없음,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,카페 엘리,127.032416,37.485785,서초구,서울특별시 서초구 강남대로39길 15-7,0,1,12,2,횃불트리니티 신학대학원대학교,746
24537,2010-09-16,1,영업/정상,NaT,50.00,카페레즈,127.041496,37.481075,서초구,서울특별시 서초구 바우뫼로41길 8,8030000,6,9,3,서울외국어대학원대학교,235
24538,2010-05-27,1,영업/정상,NaT,142.50,마켓 비노플라워,126.996156,37.498874,서초구,서울특별시 서초구 사평대로 106,12190000,5,11,1,가톨릭대학교 성의교정,839
24539,2009-10-29,1,영업/정상,NaT,62.00,까페원더,127.039916,37.484342,서초구,서울특별시 서초구 바우뫼로41길 88,7904000,4,8,3,서울외국어대학원대학교,341
